In [1]:
1+1

2

In [2]:
# 필수 라이브러리 설치 (터미널에서 실행)
# pip install -U weaviate-client python-dotenv

import os
import weaviate
import weaviate.classes.config as wc
from weaviate.classes.init import Auth
from weaviate.classes.query import Filter
from dotenv import load_dotenv

# 환경변수 로드 (선택사항)
load_dotenv()

c:\workdir\github-space-cap\GPT-study-blog\langchain_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/aggregate.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\workdir\github-space-cap\GPT-study-blog\langchain_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\workdir\github-space-cap\GPT-study-blog\langchain_project\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at v1/base_search.proto. Please update the

True

In [5]:
# OpenAI API Key 설정 (실제 키로 교체하세요)
# os.environ["OPENAI_APIKEY"] = "sk-your-actual-openai-api-key"

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Weaviate Cloud 연결 (OpenAI API Key 헤더 포함)
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers={"X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"]},
)

# 연결 상태 확인
print("Weaviate 연결 상태:", client.is_ready())

Weaviate 연결 상태: True


In [6]:
# 기존 컬렉션 삭제 (있다면)
collection_name = "ArticleCollection"
if collection_name in client.collections.list_all():
    client.collections.delete(collection_name)
    print(f"기존 {collection_name} 삭제됨")

# 새 컬렉션 생성 (최신 방식)
collection = client.collections.create(
    name=collection_name,
    vector_config=wc.Configure.Vectors.text2vec_openai(model="text-embedding-3-small"),
    generative_config=wc.Configure.Generative.openai(model="gpt-4o-mini"),
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="content", data_type=wc.DataType.TEXT),
        wc.Property(name="category", data_type=wc.DataType.TEXT),
        wc.Property(name="author", data_type=wc.DataType.TEXT),
    ],
)

print(f"컬렉션 '{collection.name}' 생성 완료!")
print("전체 컬렉션 목록:", list(client.collections.list_all().keys()))

컬렉션 'ArticleCollection' 생성 완료!
전체 컬렉션 목록: ['ArticleCollection', 'MyFirstCollection']


In [7]:
# 컬렉션 객체 가져오기
my_collection = client.collections.get(collection_name)

# 단일 객체 추가
print("=== 단일 데이터 추가 ===")
uuid1 = my_collection.data.insert(
    {
        "title": "Python 기초 가이드",
        "content": "Python 프로그래밍의 기본 개념과 문법을 소개합니다.",
        "category": "programming",
        "author": "김개발",
    }
)
print(f"단일 데이터 추가 완료, UUID: {uuid1}")

# 여러 객체 일괄 추가
print("\n=== 여러 데이터 일괄 추가 ===")
data_objects = [
    {
        "title": "AI와 머신러닝 소개",
        "content": "인공지능과 기계학습의 기본 원리를 설명합니다.",
        "category": "ai",
        "author": "박AI",
    },
    {
        "title": "웹 개발 완전 정복",
        "content": "HTML, CSS, JavaScript를 활용한 웹 개발 방법입니다.",
        "category": "web",
        "author": "이웹",
    },
    {
        "title": "데이터베이스 설계 패턴",
        "content": "효율적인 데이터베이스 설계와 최적화 기법입니다.",
        "category": "database",
        "author": "최DB",
    },
    {
        "title": "Python 고급 기법",
        "content": "Python의 고급 프로그래밍 패턴과 모범 사례입니다.",
        "category": "programming",
        "author": "김고급",
    },
]

uuids = my_collection.data.insert_many(data_objects)
print(f"{len(data_objects)}개 데이터 일괄 추가 완료!")

=== 단일 데이터 추가 ===
단일 데이터 추가 완료, UUID: 02fbc955-a2f9-4919-bf14-9d12348787ac

=== 여러 데이터 일괄 추가 ===
4개 데이터 일괄 추가 완료!


In [8]:
print("\n=== 전체 데이터 조회 ===")
response = my_collection.query.fetch_objects(limit=10)

for i, obj in enumerate(response.objects, 1):
    print(f"{i}. 제목: {obj.properties['title']}")
    print(f"   저자: {obj.properties['author']}")
    print(f"   카테고리: {obj.properties['category']}")
    print(f"   UUID: {obj.uuid}")
    print("---")


=== 전체 데이터 조회 ===
1. 제목: Python 기초 가이드
   저자: 김개발
   카테고리: programming
   UUID: 02fbc955-a2f9-4919-bf14-9d12348787ac
---
2. 제목: 웹 개발 완전 정복
   저자: 이웹
   카테고리: web
   UUID: 36479be9-ca2e-42c2-9721-37a35d70cb70
---
3. 제목: Python 고급 기법
   저자: 김고급
   카테고리: programming
   UUID: 65bb69f7-8f20-48ef-90fe-de0ac663127f
---
4. 제목: 데이터베이스 설계 패턴
   저자: 최DB
   카테고리: database
   UUID: 9b772d51-8ad2-4794-bd37-c6d80bdbb441
---
5. 제목: AI와 머신러닝 소개
   저자: 박AI
   카테고리: ai
   UUID: d1fcc417-ad7e-4cef-869f-06c709413642
---


In [9]:
print("\n=== 'programming' 카테고리 필터링 ===")
response = my_collection.query.fetch_objects(
    filters=Filter.by_property("category").equal(
        "programming"
    ),  # filters 매개변수 사용
    limit=5,
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']}")
    print(f"내용: {obj.properties['content']}")
    print(f"저자: {obj.properties['author']}")
    print("---")

print("\n=== 여러 조건 결합 (OR) ===")
response = my_collection.query.fetch_objects(
    filters=Filter.any_of(
        [
            Filter.by_property("category").equal("programming"),
            Filter.by_property("category").equal("ai"),
        ]
    ),
    limit=5,
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']} (카테고리: {obj.properties['category']})")


=== 'programming' 카테고리 필터링 ===
제목: Python 기초 가이드
내용: Python 프로그래밍의 기본 개념과 문법을 소개합니다.
저자: 김개발
---
제목: Python 고급 기법
내용: Python의 고급 프로그래밍 패턴과 모범 사례입니다.
저자: 김고급
---

=== 여러 조건 결합 (OR) ===
제목: Python 기초 가이드 (카테고리: programming)
제목: Python 고급 기법 (카테고리: programming)
제목: AI와 머신러닝 소개 (카테고리: ai)


In [14]:
response = my_collection.query.near_text(
    query="프로그래밍 개발 기술",
    limit=3,
    return_metadata=wc.query.MetadataQuery(score=True, distance=True),
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']}")
    print(f"Distance: {obj.metadata.distance}")  # distance 값 확인
    print(f"Score: {obj.metadata.score}")
    print("---")

AttributeError: module 'weaviate.classes.config' has no attribute 'query'

In [16]:
import weaviate.classes.config as wc
from weaviate.classes.query import MetadataQuery


# 하이브리드 검색 사용 (권장)
response = my_collection.query.hybrid(
    query="Python 프로그래밍",
    limit=3,
    return_metadata=MetadataQuery(score=True, explain_score=True),  # 점수 설명도 포함
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']}")
    print(f"하이브리드 점수: {obj.metadata.score}")
    print(f"점수 설명: {obj.metadata.explain_score}")
    print("---")

제목: Python 고급 기법
하이브리드 점수: 1.0
점수 설명: 
Hybrid (Result Set keyword,bm25) Document 65bb69f7-8f20-48ef-90fe-de0ac663127f: original score 1.2680364, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 65bb69f7-8f20-48ef-90fe-de0ac663127f: original score 0.47974628, normalized score: 0.7
---
제목: Python 기초 가이드
하이브리드 점수: 0.975325345993042
점수 설명: 
Hybrid (Result Set keyword,bm25) Document 02fbc955-a2f9-4919-bf14-9d12348787ac: original score 1.2680364, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 02fbc955-a2f9-4919-bf14-9d12348787ac: original score 0.46896684, normalized score: 0.67532533
---
제목: AI와 머신러닝 소개
하이브리드 점수: 0.1052849069237709
점수 설명: 
Hybrid (Result Set vector,hybridVector) Document d1fcc417-ad7e-4cef-869f-06c709413642: original score 0.21993709, normalized score: 0.10528491
---


In [19]:
print("\n=== 생성형 검색 (RAG) ===")
response = my_collection.generate.near_text(
    query="Python 프로그래밍에 대해 설명해줘",
    limit=2,
    single_prompt="다음 내용을 바탕으로 Python에 대해 간단히 설명해주세요: {content}",
)

print("🤖 AI 생성 답변:")
print(response.generative.text)
print("\n📚 참조된 문서들:")
for obj in response.objects:
    print(f"- {obj.properties['title']} (저자: {obj.properties['author']})")


=== 생성형 검색 (RAG) ===
🤖 AI 생성 답변:


📚 참조된 문서들:
- Python 기초 가이드 (저자: 김개발)
- Python 고급 기법 (저자: 김고급)


In [17]:
import weaviate.classes.config as wc
from weaviate.classes.query import MetadataQuery

# 컬렉션 가져오기
my_collection = client.collections.get("ArticleCollection")

print("=== 벡터 검색 (distance 중심) ===")
response = my_collection.query.near_text(
    query="Python 프로그래밍",
    limit=3,
    return_metadata=MetadataQuery(score=True, distance=True, certainty=True),
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']}")
    print(f"Distance: {obj.metadata.distance:.4f}")
    print(f"Score: {obj.metadata.score}")
    print(
        f"Certainty: {obj.metadata.certainty if hasattr(obj.metadata, 'certainty') else 'N/A'}"
    )
    print("---")

print("\n=== 하이브리드 검색 (score 중심) ===")
response = my_collection.query.hybrid(
    query="Python",
    limit=3,
    return_metadata=MetadataQuery(score=True, explain_score=True),
)

for obj in response.objects:
    print(f"제목: {obj.properties['title']}")
    print(f"하이브리드 점수: {obj.metadata.score:.4f}")
    if hasattr(obj.metadata, "explain_score"):
        print(f"점수 설명: {obj.metadata.explain_score}")
    print("---")

=== 벡터 검색 (distance 중심) ===
제목: Python 고급 기법
Distance: 0.5203
Score: 0.0
Certainty: 0.7398731708526611
---
제목: Python 기초 가이드
Distance: 0.5310
Score: 0.0
Certainty: 0.7344834208488464
---
제목: AI와 머신러닝 소개
Distance: 0.7801
Score: 0.0
Certainty: 0.6099685430526733
---

=== 하이브리드 검색 (score 중심) ===
제목: Python 기초 가이드
하이브리드 점수: 0.9351
점수 설명: 
Hybrid (Result Set keyword,bm25) Document 02fbc955-a2f9-4919-bf14-9d12348787ac: original score 1.2680364, normalized score: 0.3 - 
Hybrid (Result Set vector,hybridVector) Document 02fbc955-a2f9-4919-bf14-9d12348787ac: original score 0.33181828, normalized score: 0.63505894
---
제목: Python 고급 기법
하이브리드 점수: 0.7000
점수 설명: 
Hybrid (Result Set keyword,bm25) Document 65bb69f7-8f20-48ef-90fe-de0ac663127f: original score 0.6466679, normalized score: 0 - 
Hybrid (Result Set vector,hybridVector) Document 65bb69f7-8f20-48ef-90fe-de0ac663127f: original score 0.35754848, normalized score: 0.7
---
제목: AI와 머신러닝 소개
하이브리드 점수: 0.2435
점수 설명: 
Hybrid (Result Set vector,hybridV

In [20]:
from weaviate.classes.query import MetadataQuery

print("=== 생성형 검색 (RAG) - 최신 방식 ===")

# 생성형 검색 실행
response = my_collection.generate.near_text(
    query="Python 프로그래밍에 대해 설명해주세요",
    limit=2,
    single_prompt="다음 내용을 바탕으로 초보자가 이해할 수 있게 간단히 설명해주세요: {content}",
    return_metadata=MetadataQuery(score=True),
)

# 올바른 새로운 방식
if hasattr(response, "generative") and response.generative.text:
    print("🤖 AI 생성 답변:")
    print(response.generative.text)  # ✅ generative.text 사용
else:
    print("생성된 답변이 없습니다.")

# 참조 문서들
print("\n📚 참조된 문서들:")
for obj in response.objects:
    print(f"- 제목: {obj.properties['title']}")
    print(f"- 저자: {obj.properties['author']}")
    if hasattr(obj.metadata, "score"):
        print(f"- 점수: {obj.metadata.score}")
    print("---")

=== 생성형 검색 (RAG) - 최신 방식 ===
생성된 답변이 없습니다.

📚 참조된 문서들:
- 제목: Python 기초 가이드
- 저자: 김개발
- 점수: 0.0
---
- 제목: Python 고급 기법
- 저자: 김고급
- 점수: 0.0
---
